In [2]:
import numpy as np
import pandas as pd
import csv
import sys
import os

import matplotlib.pyplot as plt    
    
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import warnings
import pickle

max_iters = 501


def svm(x,y,filename):

   # Model output file name
   file = (os.path.splitext(filename))[0]
   fname = './models/svm_' + file +'/'

   # File for writing precision,recall, f-measure scores for fraud transactions
   f = open('./prf/svm_'+ file + '_prf' +'.txt' ,'w')
   f.write('precision,recall,f-score \n')

   # Stratified sampling based on Y
   X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y , test_size=0.30, random_state=42)

   # Create 15% validation set and 15% test set split
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,stratify=y_test , test_size=0.50, random_state=42)

   #Iterations
   it = 1
   
   # Run training algorithm for multiple class weights
   while it < max_iters:
       cw = {}
       cw[0] = 1
       cw[1] = it
       # Train
       print('**************************************')
       print("Iteration number  " , it)
       svm = LinearSVC(class_weight = cw, dual = False ,tol=1e-05,max_iter = 1000)
       print('Class weights ', cw)
       svm.fit(X_train,y_train)

       # Save trained model to disk
       name = fname + str(cw[1]) + '.sav'
       pickle.dump(svm, open(name, 'wb'))

       #Predict on validation data
       y_val_pred = svm.predict(X_val)
       print('Performance on validation data - Confusion matrix')
       print(confusion_matrix(y_val,y_val_pred))
   
       precision,recall,fscore,support=score(y_val,y_val_pred,average=None)
       print('Precision, Recall, F-score, Support on validation data' )
       print("Precision" , precision)
       print("Recall" , recall)
       print("F-score" , fscore)
       print("Support" , support)

       p1 = precision[1]
       r1 = recall[1]
       f1 = fscore[1]

       f.write(str(p1) +','+ str(r1) + ',' + str(f1) + '\n')    
       it += 1

   f.close()

def run():
   filename = 'full_data.csv'
   df = pd.read_csv(filename, usecols = [8,9,10,11,12,13,14] , header = 0)
   
   results = list(map(int, df['Anomaly'])) 
   print('Number of fraudulent transactions ' , sum(results))

   features = ['Transaction_Type', 'Airfare', 'Lodging', 'Meals', 'Other_Transportation', 'Other_Expenses']
   targets = ['Anomaly']

   # Separating out the features and target variables
   X = df.loc[:, features].values
   y = df.loc[:, targets].values

    
   from sklearn.preprocessing import LabelEncoder, OneHotEncoder
   labelencoder_X = LabelEncoder()
   X[:,0] = labelencoder_X.fit_transform(X[:,0])
   onehotencoder = OneHotEncoder(categorical_features = [0])
   x = onehotencoder.fit_transform(X).toarray() 

   scaler = preprocessing.StandardScaler()
   x = scaler.fit_transform(x) 
    
   y  = [i for j in y for i in j]
   
   #Ignore warnings
   warnings.filterwarnings("ignore", category=FutureWarning)

   print("**************** SVM *******************")
   svm(x,y,filename)
  
run()

Number of fraudulent transactions  8213


/home/sandisk/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


**************** SVM *******************
**************************************
Iteration number   1
Class weights  {0: 1, 1: 1}
Performance on validation data - Confusion matrix
[[953142     19]
 [   810    422]]
Precision, Recall, F-score, Support on validation data
Precision [0.9991509 0.9569161]
Recall [0.99998007 0.34253247]
F-score [0.99956531 0.50448296]
Support [953161   1232]
**************************************
Iteration number   2
Class weights  {0: 1, 1: 2}
Performance on validation data - Confusion matrix
[[953123     38]
 [   751    481]]
Precision, Recall, F-score, Support on validation data
Precision [0.99921268 0.92678227]
Recall [0.99996013 0.39042208]
F-score [0.99958627 0.54940034]
Support [953161   1232]
**************************************
Iteration number   3
Class weights  {0: 1, 1: 3}
Performance on validation data - Confusion matrix
[[953088     73]
 [   705    527]]
Precision, Recall, F-score, Support on validation data
Precision [0.99926085 0.87833333]
R

[[951961   1200]
 [   504    728]]
Precision, Recall, F-score, Support on validation data
Precision [0.99947085 0.37759336]
Recall [0.99874103 0.59090909]
F-score [0.99910581 0.46075949]
Support [953161   1232]
**************************************
Iteration number   25
Class weights  {0: 1, 1: 25}
Performance on validation data - Confusion matrix
[[951886   1275]
 [   497    735]]
Precision, Recall, F-score, Support on validation data
Precision [0.99947815 0.36567164]
Recall [0.99866235 0.59659091]
F-score [0.99907008 0.45342381]
Support [953161   1232]
**************************************
Iteration number   26
Class weights  {0: 1, 1: 26}
Performance on validation data - Confusion matrix
[[951821   1340]
 [   489    743]]
Precision, Recall, F-score, Support on validation data
Precision [0.99948651 0.35669707]
Recall [0.99859415 0.60308442]
F-score [0.99904013 0.44826546]
Support [953161   1232]
**************************************
Iteration number   27
Class weights  {0: 1, 1: 2

Performance on validation data - Confusion matrix
[[950251   2910]
 [   413    819]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956557 0.21962993]
Recall [0.996947   0.66477273]
F-score [0.99825457 0.33017537]
Support [953161   1232]
**************************************
Iteration number   49
Class weights  {0: 1, 1: 49}
Performance on validation data - Confusion matrix
[[950197   2964]
 [   413    819]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956554 0.21649485]
Recall [0.99689035 0.66477273]
F-score [0.99822615 0.32662014]
Support [953161   1232]
**************************************
Iteration number   50
Class weights  {0: 1, 1: 50}
Performance on validation data - Confusion matrix
[[950120   3041]
 [   411    821]]
Precision, Recall, F-score, Support on validation data
Precision [0.99956761 0.21258415]
Recall [0.99680956 0.6663961 ]
F-score [0.99818668 0.32234001]
Support [953161   1232]
*************************************

...

Precision, Recall, F-score, Support on validation data
Precision [0.99983846 0.03380361]
Recall [0.96752385 0.87905844]
F-score [0.98341576 0.0651037 ]
Support [953161   1232]
**************************************
Iteration number   402
Class weights  {0: 1, 1: 402}
Performance on validation data - Confusion matrix
[[922128  31033]
 [   149   1083]]
Precision, Recall, F-score, Support on validation data
Precision [0.99983844 0.03372151]
Recall [0.96744202 0.87905844]
F-score [0.98337348 0.06495142]
Support [953161   1232]
**************************************
Iteration number   403
Class weights  {0: 1, 1: 403}
Performance on validation data - Confusion matrix
[[922049  31112]
 [   149   1083]]
Precision, Recall, F-score, Support on validation data
Precision [0.99983843 0.03363876]
Recall [0.96735913 0.87905844]
F-score [0.98333066 0.06479792]
Support [953161   1232]
**************************************
Iteration number   404
Class weights  {0: 1, 1: 404}
Performance on validation 

Performance on validation data - Confusion matrix
[[920436  32725]
 [   149   1083]]
Precision, Recall, F-score, Support on validation data
Precision [0.99983815 0.03203384]
Recall [0.96566687 0.87905844]
F-score [0.98245547 0.06181507]
Support [953161   1232]
**************************************
Iteration number   426
Class weights  {0: 1, 1: 426}
Performance on validation data - Confusion matrix
[[920361  32800]
 [   149   1083]]
Precision, Recall, F-score, Support on validation data
Precision [0.99983813 0.03196293]
Recall [0.96558818 0.87905844]
F-score [0.98241474 0.06168304]
Support [953161   1232]
**************************************
Iteration number   427
Class weights  {0: 1, 1: 427}
Performance on validation data - Confusion matrix
[[920285  32876]
 [   149   1083]]
Precision, Recall, F-score, Support on validation data
Precision [0.99983812 0.0318914 ]
Recall [0.96550845 0.87905844]
F-score [0.98237346 0.06154983]
Support [953161   1232]
*********************************

Precision, Recall, F-score, Support on validation data
Precision [0.9998444  0.03076619]
Recall [0.96400713 0.88392857]
F-score [0.98159878 0.05946271]
Support [953161   1232]
**************************************
Iteration number   449
Class weights  {0: 1, 1: 449}
Performance on validation data - Confusion matrix
[[918788  34373]
 [   143   1089]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984438 0.03070893]
Recall [0.96393789 0.88392857]
F-score [0.98156287 0.05935575]
Support [953161   1232]
**************************************
Iteration number   450
Class weights  {0: 1, 1: 450}
Performance on validation data - Confusion matrix
[[918713  34448]
 [   143   1089]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984437 0.03064412]
Recall [0.9638592  0.88392857]
F-score [0.98152207 0.05923468]
Support [953161   1232]
**************************************
Iteration number   451
Class weights  {0: 1, 1: 451}
Performance on validation 

Performance on validation data - Confusion matrix
[[917260  35901]
 [   141   1091]]
Precision, Recall, F-score, Support on validation data
Precision [0.9998463  0.02949286]
Recall [0.9623348  0.88555195]
F-score [0.98073199 0.05708455]
Support [953161   1232]
**************************************
Iteration number   473
Class weights  {0: 1, 1: 473}
Performance on validation data - Confusion matrix
[[917193  35968]
 [   141   1091]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984629 0.02943954]
Recall [0.96226451 0.88555195]
F-score [0.98069548 0.05698467]
Support [953161   1232]
**************************************
Iteration number   474
Class weights  {0: 1, 1: 474}
Performance on validation data - Confusion matrix
[[917125  36036]
 [   141   1091]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984628 0.02938562]
Recall [0.96219317 0.88555195]
F-score [0.98065843 0.05688365]
Support [953161   1232]
*********************************

Precision, Recall, F-score, Support on validation data
Precision [0.99984714 0.02833493]
Recall [0.96071283 0.88636364]
F-score [0.97988941 0.05491438]
Support [953161   1232]
**************************************
Iteration number   496
Class weights  {0: 1, 1: 496}
Performance on validation data - Confusion matrix
[[915655  37506]
 [   140   1092]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984713 0.02829162]
Recall [0.96065093 0.88636364]
F-score [0.9798572  0.05483304]
Support [953161   1232]
**************************************
Iteration number   497
Class weights  {0: 1, 1: 497}
Performance on validation data - Confusion matrix
[[915596  37565]
 [   139   1093]]
Precision, Recall, F-score, Support on validation data
Precision [0.99984821 0.02827358]
Recall [0.96058903 0.88717532]
F-score [0.97982552 0.0548007 ]
Support [953161   1232]
**************************************
Iteration number   498
Class weights  {0: 1, 1: 498}
Performance on validation 